## Importing the required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Imports necessary libraries and modules.**

## Data Loading and Preprocessing

In [2]:
max_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

In [3]:
max_len = 150
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

**Loads the IMDb dataset using `imdb.load_data()` and limits the vocabulary to the top 10,000 words.**

**Pads the input sequences to a maximum length of 150.**

## Text Decoding Function

In [4]:
def sequence_to_text(sequence):
    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sequence])
    return decoded_review

In [5]:
for i in range(3):
    print(f"Example {i+1}:\n{sequence_to_text(X_train[i])}\n")

Example 1:
it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all

Example 2:
the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clot

**Defines a function `sequence_to_text` to decode integer-encoded reviews back to human-readable text.**

**Prints three examples of decoded reviews from the training data.**

##  Model Definition and Compilation

In [6]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

**Defines a Sequential model with an Embedding layer, SpatialDropout1D, Bidirectional LSTM, and Dense layers.**

**Compiles the model using binary crossentropy loss and the Adam optimizer.**

##  Model Training

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
batch_size = 32
epochs = 1
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
validation_data=(X_test, y_test), callbacks=[early_stopping])



782/782 [==============================] - 604s 757ms/step - loss: 0.4352 - accuracy: 0.7964 - val_loss: 0.3411 - val_accuracy: 0.8487


**Sets up early stopping to prevent overfitting during training.**

**Fits the model to the training data and validates on the test data.**

## Model Evaluation

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

782/782 [==============================] - 44s 56ms/step - loss: 0.3411 - accuracy: 0.8487
Accuracy: 84.87%


**Evaluates the model on the test data and prints the accuracy.**

##  Making Predictions on New Texts

In [15]:
new_texts = ["This movie is great!", "The plot is confusing.", "Amazing film with brilliant performances.", 
             "This is bad", "It is not cold outside today."]

In [16]:
# Tokenize and pad the new texts
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(new_texts)
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=max_len)

**Defines a list of new texts for sentiment analysis.**

**Tokenizes and pads the new texts using the same tokenizer and padding as the training data.**

## Displaying Predictions

In [17]:
# Make predictions
predictions = model.predict(new_padded)

1/1 [==============================] - 0s 96ms/step


In [18]:
# Display predictions
for i, text in enumerate(new_texts):
    sentiment = "Positive" if predictions[i] > 0.7 else "Negative"
    print(f"Text: '{text}'\nPredicted Sentiment: {sentiment}\n")

Text: 'This movie is great!'
Predicted Sentiment: Positive

Text: 'The plot is confusing.'
Predicted Sentiment: Negative

Text: 'Amazing film with brilliant performances.'
Predicted Sentiment: Positive

Text: 'This is bad'
Predicted Sentiment: Negative

Text: 'It is not cold outside today.'
Predicted Sentiment: Negative



**Uses the trained model to predict sentiments for the new texts.**

**Displays the original texts along with the model's predicted sentiment (Positive or Negative) based on a threshold of 0.7.**